# 파일로 데이터 읽기

In [1]:
import numpy as np
import tensorflow as tf

xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(6, 3) [[  73.   80.   75.]
 [  93.   88.   93.]
 [  89.   91.   90.]
 [  96.   98.  100.]
 [  73.   66.   70.]
 [  53.   46.   55.]] 6
(6, 1) [[ 152.]
 [ 185.]
 [ 180.]
 [ 196.]
 [ 142.]
 [ 101.]]


# Slicing, Indexing, Iterator

In [3]:
nums = np.arange(5)
print(nums)
print(nums[2:4])
print(nums[2:])
print(nums[:4])
print(nums[:-1])
nums[2:4] = [8, 9]
print(nums)

[0 1 2 3 4]
[2 3]
[2 3 4]
[0 1 2 3]
[0 1 2 3]
[0 1 8 9 4]


In [4]:
a = np.array([1, 2, 3, 4, 5])
print(a[1:3])
print(a[-1])
a[0:2] = 9
print(a)

[2 3]
5
[9 9 3 4 5]


In [5]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
print(b[:, 1])
print(b[-1])
print(b[-1, :])
print(b[-1, ...])
print(b[0:2, :])

[ 2  6 10]
[ 9 10 11 12]
[ 9 10 11 12]
[ 9 10 11 12]
[[1 2 3 4]
 [5 6 7 8]]


In [ ]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:,[-1]]

print(x_data)
print(y_data)

X = tf.placeholder(np.float32, shape=[None, 3])
Y = tf.placeholder(np.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# hypothesis
hypothesis = tf.matmul(X, W) + b

# cost function
cost = tf.reduce_mean(tf.square(hypothesis-Y))

# minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# training을 통해 cost가 최소가 되는 W, b를 구해, 임의의 X값에 대한 Y값을 추론하는 것이 목적
for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0 :
        print(step, '\ncost: ', cost_val, '\nhypothesis: ', hy_val)
   

print("Your score will be", sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))
print("Your score will be", sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

# Queue Runner

큰 파일을 읽을 때는 메모리가 부족할 수 있다. 여러개의 파일을 큐로 읽어와서 Reader로 읽어 큐에 쌓는다. 배치만큼 읽어와서 처리하면 된다.

In [ ]:
# 1. filelist를 만든다.
filename_queue = tf.train.string_input_producer(['data-01-test-score.csv'], shuffle=False, name='filename_queue')
# 2. Reader를 정의하고 value를 읽어 온다.
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
# 3. value를 처리한다.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)
# 4. batch를 만들어 읽어오게된다.
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=100)


X = tf.placeholder(np.float32, shape=[None, 3])
Y = tf.placeholder(np.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# hypothesis
hypothesis = tf.matmul(X, W) + b

# cost function
cost = tf.reduce_mean(tf.square(hypothesis-Y))

# minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 5-1. queue를 관리하다는 부분
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "\ncost: ", cost_val, "\nhypothesis: ", hy_val)
        
# 5-2. queue를 관리하다는 부분
coord.request_stop()
coord.join(threads)